## Stream flow data processing

This script reads the stream flow measurements in L/min and transforms it into m3/s. The script expects four columns: one with date and time, one with the values, one for quality control with a Y for accepted values and N for discarded values, and finally one for comments. The third column is used for filtering. NaN-values are interpolated. The data is then resampled to daily values and exported.

Joaquim Altimiras Granel, 2024

In [2]:
# Import libraries

import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
# Filter the data based on the third column and save it into a new dataframe

# Input original file path
file_path = input("Enter path for file:")

# Read the data in the file
raw_data = pd.read_excel(file_path, usecols=[0,1,2,3], names=["date_time", "flow[L/min]", "Quality", "Comment"])

# Make first column datetime and index
raw_data['date_time'] = pd.to_datetime(raw_data['date_time'])
raw_data.set_index('date_time', inplace=True)

# Convert flow from L/min to m3/s
raw_data['flow[m3/s]'] = raw_data['flow[L/min]'] / 60000  # Convert flow rate to m3/s

# Make a copy of raw_data before filtering
filtered_data = raw_data.copy()

# Filtering: If 'Quality' is not Y, then make the value a NaN-value
filtered_data.loc[filtered_data['Quality'] != 'Y', 'flow[m3/s]'] = pd.NA

# Linear interpolation: Replace NaN-values in 'flow' with linear interpolation
filtered_data['flow[m3/s]'] = filtered_data['flow[m3/s]'].interpolate(method='linear')


In [ ]:
# Plotting the filtered data (HOUR)

# Plotting the "flow[m3/s]" column from the filtered_data DataFrame
plt.figure(figsize=(10, 6)) # Sets the figure size for better readability
plt.plot(filtered_data.index, filtered_data['flow[m3/s]'], label='Flow [m3/s]', color='blue', linestyle='-')

# Adding title and labels
plt.title('Filtered Data: Flow [m3/s] Over Time (HOUR)')
plt.xlabel('Date Time')
plt.ylabel('Flow [m3/s]')
plt.xticks(rotation=45) # Rotates the x-axis labels for better readability
plt.legend()
plt.tight_layout() # Adjusts subplot params so that the subplot(s) fits in to the figure area

# Adding a grid
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Show the plot
plt.show()

In [6]:
# Resample data for daily values
# IMPORTANT: The missing days are added and interpolated here

# Resample daily
df_daily = filtered_data.copy()
daily_mean = df_daily['flow[m3/s]'].resample('D').mean()

# Ensure all days in the range are included and interpolate missing days
full_range_daily = daily_mean.reindex(pd.date_range(start=daily_mean.index.min(), end=daily_mean.index.max(), freq='D'))
interpolated_daily = full_range_daily.interpolate(method='time')


In [ ]:
# Plotting the filtered data (DAY)

# Plotting the "flow[m3/s]" column from the daily_mean DataFrame
plt.figure(figsize=(10, 6)) # Sets the figure size for better readability
plt.plot(interpolated_daily.index, interpolated_daily.values, label='Flow [m3/s]', color='blue', linestyle='-')

# Adding title and labels
plt.title('Filtered Data: Avg. Flow [m3/s] per Day Over Time')
plt.xlabel('Date Time')
plt.ylabel('Flow [m3/s]')
plt.xticks(rotation=45) # Rotates the x-axis labels for better readability
plt.legend()
plt.tight_layout() # Adjusts subplot params so that the subplot(s) fits in to the figure area

# Adding a grid
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Show the plot
plt.show()

In [ ]:
# Export the data (DAY)

# Output file path, with name added!
output_file_path = r"2_data_processing\2.6_flow_processing\filtered_data_day.xlsx"

# Export the data
daily_mean.to_excel(output_file_path, engine='openpyxl', index=True)

# Confirmation
print("Data exported successfully to:", output_file_path)